### Run a script on ssrde and return the results
- create a folder on ssrde
- (optional) copy files over
- create script to run python command
- check for output

- create folder for timestamp
- create python script
- create bash script
- copy script + python files to folder
- run sbatch

In [12]:
import paramiko
import os
from datetime import datetime
from tqdm.autonotebook import tqdm
import tempfile

In [2]:
def get_timestamp():
    return datetime.now().strftime('%d%M%Y_%H%M%S_%f')
timestamp = get_timestamp()
print(timestamp)

12382020_143817_314882


In [3]:
def get_current_folder(ssh):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('pwd')
    return ssh_stdout.read().decode("utf-8").split('\n')[0]    

### Params

In [4]:
# login info
username = 'tsainbur'
hostname = 'ssrde.ucsd.edu'

### Open SSH and transport

In [5]:
# ssh 
ssh = paramiko.SSHClient()
ssh.load_system_host_keys()
ssh.connect(hostname, username=username, password = '')

# open ftp 
sftp = ssh.open_sftp()

### create a directory for task

In [6]:
current_folder = get_current_folder(ssh)
current_folder

'/home/tsainbur'

In [7]:
process_directory = os.path.join(current_folder, 'remote_tasks' , timestamp)
process_directory

'/home/tsainbur/remote_tasks/12382020_143817_314882'

In [8]:
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('mkdir {}'.format(process_directory))
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('ls')
print(ssh_stdout.read().decode("utf-8") )

12552020_125545_574710
example_r_colab_project
miniconda3
Miniconda3-latest-Linux-x86_64.sh
nohup.out
R
remote_tasks
slurm-4569.out
slurm-4570.out
slurm-4571.out
slurm-4572.out
slurm-4573.out
tensorflowtest.py
testjob
testnvidia
testtensorflow
Untitled.ipynb
XLM



### copy file to remote

In [13]:
def copy_files_to_remote(files_dict, remote_folder):
    """
    takes in files, strings as dict. e.g :
    files_dict = {
        'file1.py': "print('hello world')"
    }
    """
    # create temporary folder
    tmpdir = tempfile.mkdtemp()
    
    # loop through files
    for fname, ftext in tqdm(files_dict.items()):
        remote_path = os.path.join(remote_folder, fname)
    
        # make temp file 
        local_path = os.path.join(tmpdir, fname)
        # wrhite text to file
        with open(local_path, "w") as tmp:
            tmp.write(ftext)

        # copy over file
        sftp.put(local_path, remote_path)
        

In [14]:
ssh_text = """#!/bin/bash
source activate ts_python3
python -c "print('Hello world!')"
"""

In [15]:
copy_files_to_remote(
    {
            "process.sh": ssh_text
    },
    process_directory
)

### run remote bash

In [16]:
remote_bash_file = os.path.join(process_directory, "process.sh")
remote_bash_file

'/home/tsainbur/remote_tasks/12382020_143817_314882/process.sh'

In [17]:
remote_path = os.path.join(process_directory, 'test.sh')

In [18]:
# make file executable
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('chmod u+rx {}'.format(remote_bash_file))
print(ssh_stdout.read().decode("utf-8") )

In [23]:
# run command in sbatch
# execute file
command = 'sbatch -p general_gpu_k40 {}'.format(remote_bash_file)
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
print(command)
print(ssh_stdout.read().decode("utf-8") )

sbatch -p general_gpu_k40 /home/tsainbur/remote_tasks/12382020_143817_314882/process.sh
Submitted batch job 15956



In [20]:
remote_bash_file

'/home/tsainbur/remote_tasks/12382020_143817_314882/process.sh'

In [21]:
kill_all = False
if kill_all:
    # run command in sbatch
    # execute file
    command = "scancel -u 'tsainbur'".format(remote_bash_file)
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
    print(command)
    print(ssh_stdout.read().decode("utf-8") )

In [150]:
sftp.close()
ssh.close()